# Weather models : 2D parameters

The aim of this notebook is to indicate how to read the 2D data from weather models.
There are 2 weather models:
* a fine-mesh french weather model called 'AROME' (spatial resolution of 0.025°)
* a large-mesh french weather model called 'ARPEGE' (spatial resolution of 0.1°)

Per model and day, for the geographic zone 'NW' (for North-West of France), you have the model run of 00h with range forecasts from 00h to 24h. The time step is 1h.

Finally, the 2D data are stored in 4 different netCDF files, according to the vertical level:

* at 2m (*2m* in the file name) : temperature (in K), dew point temperature** (in K) and relative humidity (in %)
* at 10m (*10m* in the file name): wind speed (in m.s<sup>-1</sup>), wind direction (in degrees), U and V wind components*** (in m.s<sup>-1</sup>)
* at the sea level (*P_sea_level* in the file name) : mean sea level pressure (in Pa)
* at the ground level (*PRECIP* in the file name)  : total precipitation (in kg m<sup>-2</sup> which is equivalent to mm) since the beginning of the model run 

For more information about data, cf [documentation](https://meteofrance.github.io/meteonet/english/data/weather-models/).

** : temperature for which it is needed to refresh an air volume at constant pressure and humidity to become it saturated (i.e. condensation temperature)

*** : horizontal wind speed components, U : from west to east and V : from south to north. 

**/!\ : about the total precipitation parameter, the range forecast begins to 1h and not 0h as the other parameters. It is planned in a future version to modify that parameter in order to have the total precipitation between two time steps instead.**

/!\ : Some files in the model files are incomplete/corrupted. It comes from the database of our corporation. There are archiving anomalies. We are working on a new version of MeteoNet and will take into account these anomalies. Meanwhile, you can detect these anomalies by checking the file sizes. For a given file category (zone, level, model type), the files must have exactly the same size. So if some files are smaller than the other one, then these files have anomalies. This has been reported in kaggle in this discussion https://www.kaggle.com/katerpillar/meteonet/discussion/157206 and in Github (https://github.com/meteofrance/meteonet/issues/14).

# Note

<font size="4.5">To use <span style="color:blue">**Cartopy**</span>, a library to plot data with basemaps (see cells below), it is necessary to <span style="color:red">activate the internet connection</span> of that notebook (in edit mode, you can find on the right column, in the *Settings* section, a row entitled *Internet*, put the slider bar on **on**).  </font>

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# Input data files are available in the "../input/" directory.
# Any results you write to the current directory are saved as output.

## Parameters definition

Select the area, model, date, level and parameter you want to study :

In [ ]:
zone = "NW"     #geographic zone (NW or SE)
model = 'arpege' #weather model (arome or arpege)
MODEL = 'ARPEGE' #weather model (AROME or ARPEGE)
level = '2m'      #vertical level (2m, 10m, P_sea_level or PRECIP)
date = dt.datetime(2016, 2, 14,0,0) # Day example 
#parameter name in the file (cf cells below to know the parameter names -> exploration of metadata)
if level == '2m':
    param = 't2m'
elif level == '10m':
    param = 'u10'
elif level == 'PRECIP':
    param = 'tp'
else:
    param = 'msl'

## Loading data using parameters 

In [ ]:
directory = '/kaggle/input/meteonet/' + zone + '_weather_models_2D_parameters_' + str(date.year) + str(date.month).zfill(2) + '/' + str(date.year) + str(date.month).zfill(2) + '/'
fname = directory + f'{MODEL}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'
data = xr.open_dataset(fname)  

## Quick field visualisation
The following comand shows the weather forecast at step 0 (analyse)  6/12/24 for temperature.
As the run chosed in parameter is the midnight run, the temperature at step 12 (figure 3) is the midday temperature. 
You can see that this temperature is higher than midnight (fig 1 and 4) or at 6AM (figure 2)

In [ ]:
data.isel(step=[0, 6, 12, 23])[param].plot(x='longitude',
                                           y='latitude',
                                           col='step',
                                           col_wrap=2)

## Exploration of metadata 

Overview of the data :

In [ ]:
data

Get the information about coordinates (latitude and longitude):

In [ ]:
coord = 'longitude'
data[coord]

In [ ]:
data[coord].units

In [ ]:
data[coord].values[0:10]

Get the information about the run date and the different range forecasts:

In [ ]:
run_date = data['time']
#run_date.values     #get the values
run_date

In [ ]:
range_forecasts_dates = data['valid_time']
range_forecasts_dates

Get the information about the vertical level:

In [ ]:
if (level =='2m' or level == '10m'):
    level_name = 'heightAboveGround'
elif (level =='P_sea_level'):
    level_name = 'meanSea'
else:
    level_name = 'surface'
info_level = data[level_name]
info_level

Get the information about one parameter: 
the parameter names in the GRIB file are indicated in the field *Data variables* (cf print(data) above)

In [ ]:
d = data[param]     #param : parameter name defined at the beginning of the Notebook 
d_vals=d.values     #get the values
###examples to get the information from attributes
#d.units                      #unit
#d.long_name                      #long name
d

The structure of the parameter (3 dimensions):
* number of steps or range forecasts
* number of points in latitude
* number of points in longitude  

In [ ]:
d_vals.shape

## Plotting data with Cartopy

Plot the parameter values for 1 given time step with Cartopy:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#index for the studied time step
step = 0                

In [ ]:
# Coordinates of studied area boundaries (in °N and °E)
lllat = 46.25  #lower left latitude
urlat = 51.896  #upper right latitude
lllon = -5.842  #lower left longitude
urlon = 2  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

fig=plt.figure(figsize=(9,10))

# Select projection
ax = plt.axes(projection=ccrs.PlateCarree())

#plot the data and the background map (coastlines and borders)
img = ax.imshow(d_vals[step,:,:], interpolation='none', origin='upper', extent=extent)
ax.coastlines(resolution='50m', linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale('50m'))


plt.colorbar(img, orientation= 'horizontal').set_label(d.long_name+ ' (in '+d.units+ ')')
plt.title(model +" model - "+str(d['valid_time'].values[step])+" - " +zone + " zone")
plt.show()